# Initialization

This notebook is forked and inspired from the [BELKA 1DCNN Starter](https://www.kaggle.com/code/ahmedelfazouan/belka-1dcnn-starter-with-all-data/notebook).

This original notebook attempted to encoded the smiles of all the train & test set and saved it locally, this may take up to 1 hour on TPU for each fold. While the original notebook only utilizes one fold, there is only 1/15 data used for training, due to the limit of computational power. Therefore, I pre-trained one model for each fold and store the model weights locally so that the prediction results could be combined to make better LB score. The models are stored [here](https://www.kaggle.com/datasets/hugowjd/1dcnn-models-for-belka-competition).

The encoded data is stored [here](https://www.kaggle.com/datasets/ahmedelfazouan/belka-enc-dataset) , 

How to improve :
* Change the fold size (better machine may deal with smaller fold number)
* Try another model like Transformer, or LSTM.
* Train for more epochs for each folds.
* Add more features like a one hot encoding of bb2 or bb3.
* And of ensembling with other models.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install fastparquet -q

In [ ]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [ ]:
class CFG:

    PREPROCESS = False
    EPOCHS = 15
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05
    # Number of folds
    NBR_FOLDS = 15
    PRETRAINED = True
    # Only the first fold selected
    SELECTED_FOLDS = []
    EXIST_MODELS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]

    SEED = 2024

In [ ]:
import tensorflow as tf
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

In [ ]:
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except tf.errors.NotFoundError:
    print("Not on TPU")

# Preprocessing

In [ ]:
if CFG.PREPROCESS:
    enc = {'l': 1, 'y': 2, '@': 3, '3': 4, 'H': 5, 'S': 6, 'F': 7, 'C': 8, 'r': 9, 's': 10, '/': 11, 'c': 12, 'o': 13,
           '+': 14, 'I': 15, '5': 16, '(': 17, '2': 18, ')': 19, '9': 20, 'i': 21, '#': 22, '6': 23, '8': 24, '4': 25, '=': 26,
           '1': 27, 'O': 28, '[': 29, 'D': 30, 'B': 31, ']': 32, 'N': 33, '7': 34, 'n': 35, '-': 36}
    train_raw = pd.read_parquet('/kaggle/input/leash-BELKA/train.parquet')
    smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values
    assert (smiles!=train_raw[train_raw['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
    assert (smiles!=train_raw[train_raw['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0
    def encode_smile(smile):
        tmp = [enc[i] for i in smile]
        tmp = tmp + [0]*(142-len(tmp))
        return np.array(tmp).astype(np.uint8)

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    train = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    train['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
    train['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
    train['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values
    train.to_parquet('train_enc.parquet')

    test_raw = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
    smiles = test_raw['molecule_smiles'].values

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    test = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    test.to_parquet('test_enc.parquet')

else:
    train = pd.read_parquet('/kaggle/input/belka-enc-dataset/train_enc.parquet')
    test = pd.read_parquet('/kaggle/input/belka-enc-dataset/test_enc.parquet')

In [ ]:
print("Shape of Train set:", train.shape)
train.head(3)

In [ ]:
print("Shape of Test set:", test.shape)
test.head(3)

# Modeling

In [ ]:
# 1D-CNN model
def OneDCNN_model():
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 32
        hidden_dim = 128

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

# Train & Inference

In [ ]:
%time
FEATURES = [f'enc{i}' for i in range(142)] # The first 142 encoded columns as features
TARGETS = ['bind1', 'bind2', 'bind3'] # Three types of binds as targets
# 15 fold -> only train 1/15 of the entire dataset
# If we got better machine, we may change NBR_FOLDS smaller
skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42) 
                                                                                    

all_preds = []
if CFG.PRETRAINED:
    for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):
        print(f"Working on fold {fold}")
#         X_val = train.loc[valid_idx, FEATURES]
#         y_val = train.loc[valid_idx, TARGETS]

        model = OneDCNN_model()
        model.load_weights(f"/kaggle/input/1dcnn-models-for-belka-competition/model-{fold}.h5")
#         oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
#         print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))
        preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
        all_preds.append(preds)
else:

    for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):

        if fold in CFG.EXIST_MODELS:
            print(f"Working on fold {fold}")
#             X_val = train.loc[valid_idx, FEATURES]
#             y_val = train.loc[valid_idx, TARGETS]

            model = OneDCNN_model()
            model.load_weights(f"/kaggle/input/1dcnn-models-for-belka-competition/model-{fold}.h5")
#             oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
#             print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))
            preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
            all_preds.append(preds)
            continue;

        if fold in CFG.SELECTED_FOLDS:
            print(f"Working on fold {fold}")
            X_train = train.loc[train_idx, FEATURES]
            y_train = train.loc[train_idx, TARGETS]
            X_val = train.loc[valid_idx, FEATURES]
            y_val = train.loc[valid_idx, TARGETS]

            es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
            checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
                                                                save_best_only=True, save_weights_only=True,
                                                            mode='min')
            reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
            model = OneDCNN_model()
            history = model.fit(
                    X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=CFG.EPOCHS,
                    callbacks=[checkpoint, reduce_lr_loss, es],
                    batch_size=CFG.BATCH_SIZE,
                    verbose=1,
                )
            model.load_weights(f"model-{fold}.h5")
            oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
            print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))

            preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
            all_preds.append(preds)

preds = np.mean(all_preds, 0)

In [ ]:
print(f"Shape of Predictions: {len(preds)} * {len(preds[0])}")

# Plot

In [ ]:
import matplotlib.pyplot as plt
# Assuming 'history' is the object returned from model.fit()
if CFG.PRETRAINED:
    print("Pretrained -> no plot")
else:
    train_loss = history.history['loss']
    val_loss = history.history.get('val_loss', [])  # Use get to avoid errors if validation loss is not available

    train_prec = history.history['avg_precision']
    val_prec = history.history.get('val_avg_precision', [])  # Similarly for validation accuracy
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train')
    if val_loss:
        plt.plot(val_loss, label='Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(train_prec, label='Train')
    if val_prec:
        plt.plot(val_prec, label='Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')

    plt.tight_layout()
    plt.show()

# Submission

In [ ]:
tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)